In [1]:
#import GPy
import time
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
#from xgboost import plot_importance
from numpy import loadtxt
#from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
import pickle
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.ensemble import GradientBoostingClassifier
#from pycaret.classification import *

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
import warnings

warnings.filterwarnings("ignore")  # ignore DeprecationWarnings from tensorflow

import matplotlib.pyplot as plt

%matplotlib inline

import gpflow

from gpflow.utilities import print_summary, set_trainable
from gpflow.ci_utils import ci_niter

#from multiclass_classification import plot_posterior_predictions, colors

# reproducibility:
np.random.seed(0)
tf.random.set_seed(123)
import warnings


warnings.warn("ignore")

In [ ]:
### try with following versions of input data

In [4]:
X_train = np.load('X_train_afr_manual_w2v.npy')
y_train = np.load('y_train.npy').astype(int)
X_test = np.load('X_test_afr_manual_w2v.npy')
y_test = np.load('y_test.npy').astype(int)

In [5]:
# X_train = np.load('X_train_afr_google_w2v.npy')
# y_train = np.load('y_train.npy').astype(int)
# X_test = np.load('X_test_afr_google_w2v.npy')
# y_test = np.load('y_test.npy').astype(int)

In [6]:
y_train=y_train[:,np.newaxis]
y_test=y_test[:,np.newaxis]

In [7]:
data=(X_train,y_train)

# Using Convolutional Kernel

In [10]:
### slecting 244 indiced points 
X=X_train[::1000]
Y=y_train[::1000]

In [11]:
X.shape

(244, 50)

In [12]:
data=(X,Y)

In [15]:
warnings.warn("ignore")

In [16]:
IMAGE_SHAPE = [1,50] #[1,50] for our own w2vec model and  [1,300] for  google word2vec model
f64 = lambda x: np.array(x, dtype=np.float64)
positive_with_min = lambda: tfp.bijectors.AffineScalar(shift=f64(1e-4))(tfp.bijectors.Softplus())
constrained = lambda: tfp.bijectors.AffineScalar(shift=f64(1e-4), scale=f64(100.0))(
    tfp.bijectors.Sigmoid()
)
max_abs_1 = lambda: tfp.bijectors.AffineScalar(shift=f64(-2.0), scale=f64(4.0))(
    tfp.bijectors.Sigmoid()
)

patch_shape = [1, 5] 
### vary patch/filter size and check the final accuracy 
##(when filter dimensions are changed , accordingly change reshape dimension below)
conv_k = gpflow.kernels.Convolutional(gpflow.kernels.SquaredExponential(), IMAGE_SHAPE, patch_shape)
conv_k.base_kernel.lengthscales = gpflow.Parameter(1.0, transform=positive_with_min())
conv_k.base_kernel.lengthscales = gpflow.Parameter(1.0)
#Weight scale and variance are non-identifiable. We also need to prevent variance from shooting off crazily.
conv_k.base_kernel.variance = gpflow.Parameter(1.0, transform=constrained())
conv_k.weights = gpflow.Parameter(conv_k.weights.numpy(), transform=max_abs_1())
conv_f = gpflow.inducing_variables.InducingPatches(
    np.unique(conv_k.get_patches(X).numpy().reshape(-1, 5), axis=0)
)

#### case 1 : substitute X  with inducing points
#### case 2 : substitute X with total X_train

In [17]:
conv_m = gpflow.models.SVGP(conv_k, gpflow.likelihoods.Bernoulli(), conv_f)

In [18]:
set_trainable(conv_m.inducing_variable, False)
set_trainable(conv_m.kernel.base_kernel.variance, False)
set_trainable(conv_m.kernel.weights, False)

In [24]:
%%time
conv_training_loss_closure = conv_m.training_loss_closure(data, compile=True)
conv_elbo = lambda: -conv_training_loss_closure().numpy()
print("conv elbo before training: %.4e" % conv_elbo())

conv elbo before training: -1.8634e+02
CPU times: user 34.2 s, sys: 4.87 s, total: 39.1 s
Wall time: 21 s


In [25]:
%%time
start_time = time.time()
res = gpflow.optimizers.Scipy().minimize(
    conv_training_loss_closure,
    variables=conv_m.trainable_variables,
    method="l-bfgs-b",
    options={"disp": True, "maxiter": 100 },
)
print(f"{res.nfev / (time.time() - start_time):.3f} iter/s")

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
0.013 iter/s
CPU times: user 7h 29min 12s, sys: 14min 44s, total: 7h 43min 56s
Wall time: 1h 32min 22s


In [ ]:
### accuracy of first 100 data points  

In [26]:
train_acc = np.mean((conv_m.predict_y(X)[0] > 0.5).numpy().astype("float") == Y)
test_acc = np.mean((conv_m.predict_y(X_test[0:100])[0] > 0.5).numpy().astype("float") == y_test[0:100])
print(f"Train acc: {train_acc * 100}%\nTest acc : {test_acc*100}%")
print("conv elbo after training: %.4e" % conv_elbo())

Train acc: 85.65573770491804%
Test acc : 82.0%
conv elbo after training: -1.0294e+02


In [ ]:
### Accuracy calculation for entire dataset 
### as we cant run entire dataset ata a time , calculate acc for 1000 points ata atime and average them

In [ ]:

%%time
s1=1
s2=1000
final_test=0
acc_list=[]
while s2 < len(X_train):
    
    test_acc = np.mean((conv_m.predict_y(X_test[s1:s2])[0] > 0.5).numpy().astype("float") == y_test[s1:s2])
    #print(test_acc)
    #print(f"Test acc : {test_acc*100}%")
    acc_list.append(test_acc)
    #print(acc_list)
    s1=s2
    s2=s2+1000

In [135]:
acc_list_test = np.load('acc_list_test.npy')
acc_list_test = acc_list_test[~(np.isnan(acc_list_test))]
print("test accuracy",np.mean(acc_list_test))

test accuracy 0.8408509189957573


In [1]:
#### simialry calculate train accuracy 


In [ ]:
%%time
s1=1
s2=1000
final_test=0
acc_tr_list=[]
while s2 < len(X_train):
    
    train_acc = np.mean((conv_m.predict_y(X_train[s1:s2])[0] > 0.5).numpy().astype("float") == y_train[s1:s2])
    #print(test_acc)
    #print(f"Test acc : {test_acc*100}%")
    acc_tr_list.append(train_acc)
    #print(acc_list)
    s1=s2
    s2=s2+1000
#acc=(final_test/s2)*100
#print(f"Test acc : {test_acc*100}%")
#print("conv elbo after training: %.4e" % conv_elbo())

In [ ]:
np.save("acc_list_train.npy",acc_tr_list)
print(s2)

In [136]:
acc_list_train = np.load('acc_list_train.npy')
acc_list_train = acc_list_train[~(np.isnan(acc_list_train))]
print("train accuracy",np.mean(acc_list_train))

train accuracy 0.8406165713038141
